<p>Dieses Nodebook schaut mal was passiert wenn wir die erhöhte Regelleistung aufrung von Kraftwerksausfällen filtern </p>

In [1]:
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
daten = pd.read_csv("../Ergebnisse/Modellierungen/V02/modellierung_02.csv", index_col=0)
outages = pd.read_csv("../../outages_unplanned aggregated alle.csv", index_col=0)

daten_und_outages = daten.join(outages["Delta Summe"])
daten_und_outages

,Deutschland (Positiv),Deutschland (Negativ),Solar,Wind Offshore,Wind Onshore,Residuallast,wind_speed_100m,wind_gusts_10m,cloudcover,temperature_2m,...,_sin_year,_cos_year,_sin_week,_cos_week,_sin_day,_cos_day,load forecast error Deutschland,gradient Deutschland,Feiertagsindex,Delta Summe
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-06-30 22:00:00+00:00,390.620,350.816,3.0,1317.0,13862.0,37257.0,22.50,22.700,100.0,18.100,...,0.025818,-0.999667,0.433884,-0.900969,-0.500000,0.866025,1044.77,-107.77,False,0.0
2022-06-30 22:15:00+00:00,162.736,263.656,3.0,1459.0,13702.0,37019.0,22.65,22.775,100.0,17.300,...,0.025818,-0.999667,0.433884,-0.900969,-0.442289,0.896873,230.69,-1092.17,False,0.0
2022-06-30 22:30:00+00:00,928.544,676.444,3.0,1480.0,13340.0,36835.0,22.80,22.850,100.0,16.500,...,0.025818,-0.999667,0.433884,-0.900969,-0.382683,0.923880,171.95,-695.35,False,0.0
2022-06-30 22:45:00+00:00,1212.684,825.452,3.0,1428.0,12910.0,36779.0,22.95,22.925,100.0,15.700,...,0.025818,-0.999667,0.433884,-0.900969,-0.321439,0.946930,122.87,-392.45,False,0.0
2022-06-30 23:00:00+00:00,1402.408,963.996,3.0,1352.0,12591.0,37140.0,23.10,23.000,100.0,14.900,...,0.025818,-0.999667,0.433884,-0.900969,-0.258819,0.965926,-31.08,-563.35,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-17 21:45:00+00:00,393.812,397.516,7.0,2849.0,2403.0,40560.0,11.10,18.175,100.0,-2.550,...,0.288482,0.957485,-0.433884,-0.900969,-0.555570,0.831470,0.00,0.00,False,NaN
2025-01-17 22:00:00+00:00,470.056,646.360,6.0,2893.0,2499.0,41371.0,10.40,18.000,100.0,-2.600,...,0.288482,0.957485,-0.433884,-0.900969,-0.500000,0.866025,0.00,0.00,False,NaN
2025-01-17 22:15:00+00:00,786.280,721.776,6.0,2920.0,2553.0,40780.0,9.55,17.375,100.0,-2.525,...,0.288482,0.957485,-0.433884,-0.900969,-0.442289,0.896873,0.00,0.00,False,NaN


In [13]:
print(f"Korrelation positiv: {daten_und_outages["Deutschland (Positiv)"].corr(daten_und_outages["Delta Summe"])}")
print(f"Korrelation negativ: {daten_und_outages["Deutschland (Negativ)"].corr(daten_und_outages["Delta Summe"])}")



Korrelation positiv: 0.011582139725375843
Korrelation negativ: 0.016595674859228207


In [ ]:
#Daten ohne das betroffene Zeitfenster

daten_und_outages_ohne_ausfälle = daten_und_outages[(daten_und_outages["Delta Summe"] <= 0)]
daten_und_outages_ohne_ausfälle.describe()



,Deutschland (Positiv),Deutschland (Negativ),Solar,Wind Offshore,Wind Onshore,Residuallast,wind_speed_100m,wind_gusts_10m,cloudcover,temperature_2m,...,precipitation,_sin_year,_cos_year,_sin_week,_cos_week,_sin_day,_cos_day,load forecast error Deutschland,gradient Deutschland,Delta Summe
count,84259.000000,84259.000000,84259.000000,84259.000000,84259.000000,84259.000000,84259.000000,84259.000000,84259.000000,84259.000000,...,84259.000000,84259.000000,84259.000000,84259.000000,84259.000000,84259.000000,8.425900e+04,84259.000000,84259.000000,84259.0
mean,692.687285,654.549510,6550.966389,2819.564379,12524.673614,29568.407624,26.195499,31.224260,66.621026,12.820181,...,0.136530,-0.125172,-0.002567,-0.006971,-0.000164,0.002375,6.326499e-03,-427.209452,0.806034,0.0
std,396.534903,381.189872,10229.356344,1864.455141,9909.048509,9860.612238,11.181459,13.776383,38.000828,7.733757,...,0.490557,0.694718,0.708310,0.707961,0.706225,0.707011,7.071789e-01,2549.054065,656.105528,0.0
min,0.000000,0.000000,1.000000,0.000000,42.000000,12449.000000,0.400000,1.400000,0.000000,-8.400000,...,0.000000,-0.999991,-0.999963,-0.974928,-0.900969,-1.000000,-1.000000e+00,-11376.940000,-6238.470000,0.0
25%,409.880000,383.238000,4.000000,1143.000000,4664.000000,21746.000000,18.100000,20.975000,30.000000,7.325000,...,0.000000,-0.790946,-0.708627,-0.781831,-0.900969,-0.707107,-7.071068e-01,-2084.100000,-433.100000,0.0
50%,613.296000,581.892000,67.000000,2625.000000,9724.000000,28438.000000,25.400000,29.725000,87.500000,12.600000,...,0.000000,-0.255353,-0.012910,0.000000,-0.222521,0.000000,-1.608123e-16,-427.250000,-67.410000,0.0
75%,886.682000,845.210000,10324.500000,4387.000000,18258.000000,35961.000000,33.200000,39.875000,100.000000,18.350000,...,0.000000,0.559589,0.702527,0.781831,0.623490,0.707107,7.071068e-01,1239.610000,344.070000,0.0
max,3621.212000,4350.640000,47066.000000,7640.000000,48607.000000,62157.000000,76.850000,96.500000,100.000000,36.600000,...,20.800000,0.999991,1.000000,0.974928,1.000000,1.000000,1.000000e+00,9885.020000,6786.280000,0.0


In [26]:
#Daten ohne das Betroffene Zeitfenster und t + 1 und t - 1
daten_und_outages = daten_und_outages.reset_index()
mask_zu_löschen = daten_und_outages["Delta Summe"] > 0

indices = set()
for idx in daten_und_outages.index[mask_zu_löschen]:
    indices.update([idx, idx + 1, idx + 2, idx + 3, idx + 4])

# Schritt 3: Entferne ungültige Indizes (z. B. -1, len(df))
indices = [i for i in indices if i in daten_und_outages.index]

# Schritt 4: Zeilen löschen
df_bereinigt = daten_und_outages.drop(index=indices)
df_bereinigt.set_index("timestamp")

daten_und_outages = daten_und_outages.set_index("timestamp")

In [27]:
df_bereinigt.to_csv("modellierung_02_ohne_ausfälle_+1_bis_+4.csv", index=True)
